In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
import random
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from node2vec import Node2Vec

In [ ]:
# Load Karate Club Graph
graph = nx.karate_club_graph()
nodes = list(graph.nodes())

In [ ]:
# Parameters
num_walks = 10  # Number of random walks per node
walk_length = 10  # Length of each random walk
embedding_dim = 128  # Embedding size
num_epochs = 50
learning_rate = 0.01

In [ ]:
# Generate random walks
def generate_random_walks(graph, num_walks, walk_length):
    walks = []
    for _ in range(num_walks):
        for node in graph.nodes():
            walk = [node]
            for _ in range(walk_length - 1):
                neighbors = list(graph.neighbors(walk[-1]))
                if neighbors:
                    walk.append(random.choice(neighbors))
                else:
                    break
            walks.append(walk)
    return walks

walks = generate_random_walks(graph, num_walks, walk_length)

In [ ]:
# Convert walks to training pairs
positive_pairs = []
for walk in walks:
    for i, node in enumerate(walk):
        for j in range(max(0, i - 2), min(len(walk), i + 3)):
            if i != j:
                positive_pairs.append((node, walk[j]))

In [ ]:
# Negative sampling
all_nodes = set(graph.nodes())
negative_pairs = []
for node1, node2 in positive_pairs:
    while True:
        neg_sample = random.choice(nodes)
        if neg_sample != node1 and neg_sample not in graph.neighbors(node1):
            negative_pairs.append((node1, neg_sample))
            break

In [ ]:
# Prepare training data
train_data = positive_pairs + negative_pairs
train_labels = [1] * len(positive_pairs) + [0] * len(negative_pairs)

# Convert to PyTorch tensors
train_data = torch.tensor(train_data, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.float32)

In [ ]:
# Define Neural Network
class NodeEmbeddingNN(nn.Module):
    def __init__(self, num_nodes, embedding_dim):
        super(NodeEmbeddingNN, self).__init__()
        self.embeddings = nn.Embedding(num_nodes, embedding_dim)

    def forward(self, node_pairs):
        node1 = self.embeddings(node_pairs[:, 0])
        node2 = self.embeddings(node_pairs[:, 1])
        return (node1 * node2).sum(dim=1)

In [ ]:
# Model Initialization
model = NodeEmbeddingNN(len(nodes), embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.BCEWithLogitsLoss()

In [ ]:
# Training Loop
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(train_data)
    loss = loss_function(outputs, train_labels)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# Extract learned embeddings
embeddings = model.embeddings.weight.detach().numpy()

In [ ]:
# t-SNE Visualization
def plot_embeddings(embeddings, graph):
    tsne = TSNE(n_components=2)
    embeddings_2d = tsne.fit_transform(embeddings)
    plt.figure(figsize=(8, 6))
    for i, label in enumerate(graph.nodes()):
        plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], label=label)
    plt.show()

In [ ]:
plot_embeddings(embeddings, graph)

In [ ]:
# Compare with DeepWalk
print("Training DeepWalk...")
deepwalk_model = Word2Vec(sentences=walks, vector_size=embedding_dim, window=5, min_count=1, sg=1, workers=4)
deepwalk_embeddings = np.array([deepwalk_model.wv[str(node)] for node in nodes])
plot_embeddings(deepwalk_embeddings, graph)

In [ ]:
# Compare with Node2Vec
print("Training Node2Vec...")
node2vec = Node2Vec(graph, dimensions=embedding_dim, walk_length=walk_length, num_walks=num_walks, workers=4)
node2vec_model = node2vec.fit(window=5, min_count=1, sg=1)
node2vec_embeddings = np.array([node2vec_model.wv[str(node)] for node in nodes])
plot_embeddings(node2vec_embeddings, graph)